Pagination in Django REST Framework (DRF) is a crucial feature for handling large datasets efficiently. It allows you to divide your data into smaller, more manageable chunks, improving performance and user experience. Here's a breakdown of how it works and common pagination styles:

**Key Concepts**

* **`Pagination` Classes:** DRF provides abstract `Pagination` classes that you inherit from and configure. You can use the built-in ones or create custom implementations.
* **`page` Parameter:** Most paginated responses will include a `page` parameter to indicate the current page number.
* **`page_size` Parameter:** The `page_size` parameter determines how many items are displayed per page.
* **Response Structure:** Paginated responses usually include:
    * `count`: Total number of items.
    * `next`: URL to the next page (or `null` if there isn't one).
    * `previous`: URL to the previous page (or `null` if there isn't one).
    * `results`: The list of items on the current page.

**Built-in Pagination Styles**

DRF comes with several pre-built pagination styles:

1. **`PageNumberPagination`:** (Most common)
   * Uses standard page numbers.
   * URL parameters: `page`, `page_size`
   * Example URL: `/api/users/?page=2&page_size=10`

2. **`LimitOffsetPagination`:**
   * Uses a limit and offset to retrieve data.
   * URL parameters: `limit`, `offset`
   * Example URL: `/api/users/?limit=10&offset=20`

3. **`CursorPagination`:**
   * Uses an opaque cursor (usually an encoded ID or timestamp) for pagination.
   * Ideal for very large datasets because it avoids performance issues with large `offset` values.
   * URL parameter: `cursor`
   * Example URL: `/api/users/?cursor=eyJpZCI6IDUzMH0=`


**How to Use Pagination**


1. **Define a `Pagination` Class:** Choose the pagination style you want and configure its settings.


In [ ]:

# myapp/pagination.py
from rest_framework import pagination

class CustomPageNumberPagination(pagination.PageNumberPagination):
  page_size = 10  # Default page size
  page_size_query_param = 'page_size' # Allow clients to override
  max_page_size = 100 # Maximum page size allowed

from rest_framework import pagination

class CustomLimitOffsetPagination(pagination.LimitOffsetPagination):
  default_limit = 10
  max_limit = 100

# myapp/pagination.py'

from rest_framework import pagination
   
class CustomCursorPagination(pagination.CursorPagination):
  page_size = 10
  ordering = '-created_at' # Ordering field is required



**Set Pagination Globally (Recommended):**  Configure pagination in your `settings.py`.


   # settings.py
   REST_FRAMEWORK = {
       'DEFAULT_PAGINATION_CLASS': 'myapp.pagination.CustomPageNumberPagination',
       # Or: 'DEFAULT_PAGINATION_CLASS': 'myapp.pagination.CustomLimitOffsetPagination',
       # Or: 'DEFAULT_PAGINATION_CLASS': 'myapp.pagination.CustomCursorPagination',
   }
   ```

3. **Set Pagination per View (Optional):** You can override the global settings for specific views using `pagination_class`.

   ```python
   # myapp/views.py
   from rest_framework import generics
   from myapp.models import User
   from myapp.serializers import UserSerializer
   from myapp.pagination import CustomLimitOffsetPagination # For example
   #from myapp.pagination import CustomPageNumberPagination

   class UserListView(generics.ListAPIView):
       queryset = User.objects.all()
       serializer_class = UserSerializer
       pagination_class = CustomLimitOffsetPagination
       #pagination_class = CustomPageNumberPagination
   ```

**Example Code with `PageNumberPagination`**

```python
# myapp/models.py
from django.db import models

class User(models.Model):
    username = models.CharField(max_length=100)
    email = models.EmailField()
    created_at = models.DateTimeField(auto_now_add=True)

# myapp/serializers.py
from rest_framework import serializers
from .models import User

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'username', 'email', 'created_at']

# myapp/views.py
from rest_framework import generics
from .models import User
from .serializers import UserSerializer
from .pagination import CustomPageNumberPagination  # Import CustomPagination

class UserListView(generics.ListAPIView):
    queryset = User.objects.all()
    serializer_class = UserSerializer
    pagination_class = CustomPageNumberPagination # Set the pagination
    
# myapp/urls.py
from django.urls import path
from .views import UserListView
urlpatterns = [
  path("users/", UserListView.as_view(), name="user-list")
]
```

**Example Response with `PageNumberPagination`:**

```json
{
  "count": 120,
  "next": "/api/users/?page=2&page_size=10",
  "previous": null,
  "results": [
    {
      "id": 1,
      "username": "user1",
      "email": "user1@example.com",
      "created_at": "2023-11-20T10:00:00Z"
    },
    {
       "id": 2,
       "username": "user2",
       "email": "user2@example.com",
       "created_at": "2023-11-20T11:00:00Z"
    },
    ...
  ]
}
```



**Important Considerations**

* **Database Performance:** When working with large datasets, make sure your database queries are optimized for pagination (e.g., using indexes on relevant fields).
* **Client-Side Handling:** Clients need to handle the paginated response, extracting the data and using `next` and `previous` URLs for subsequent page requests.
* **Custom Logic:** You can further customize pagination behavior by:
    * Overriding the `get_paginated_response` method in your custom pagination classes.
    * Adding custom URL query parameters for filtering or sorting within a page.
* **Filtering & Sorting:** Pagination often works hand-in-hand with filtering and sorting. Ensure your view handles these operations together.
* **Cursor vs. Offset:** If you anticipate very large datasets, favor `CursorPagination` over `LimitOffsetPagination` for better performance. `PageNumberPagination` uses offset so has similar performance implications.

**In summary:**

1. Define your pagination class.
2. Either set a default one in settings or set on a view-by-view basis.
3. Clients should use `next` and `previous` URLs to browse the data.
4. Optimize database queries.

By implementing pagination effectively, you can create more performant and user-friendly APIs.
